In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
status_df = pd.read_csv("../../Data/filtered_status.csv", parse_dates=['time'])

In [3]:
status_df

,station_id,bikes_available,docks_available,time
0,2,2,25,2013-08-29 12:06:01
1,2,2,25,2013-08-29 12:07:01
2,2,2,25,2013-08-29 12:08:01
3,2,2,25,2013-08-29 12:09:01
4,2,2,25,2013-08-29 12:10:01
...,...,...,...,...
71977905,84,8,7,2015-08-31 23:55:02
71977906,84,8,7,2015-08-31 23:56:01
71977907,84,8,7,2015-08-31 23:57:02
71977908,84,8,7,2015-08-31 23:58:02


In [4]:
status_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71977910 entries, 0 to 71977909
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   station_id       int64         
 1   bikes_available  int64         
 2   docks_available  int64         
 3   time             datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 2.1 GB


In [4]:
# time of the first record
min(status_df['time'])

Timestamp('2013-08-29 12:06:01')

In [3]:
# add day column to the dataframe
status_df['day'] = status_df['time'].dt.date

In [26]:
# calculate how often stations are (almost) empty/full

for ID in [70, 76, 34, 72, 75, 69, 50, 60, 73, 82, 84]: #arbitrary set of stations
    IDdf = status_df[status_df['station_id']==ID]

    length = []
    full = []
    almost_full = []
    normal = []
    almost_empty = []
    empty = []

    for group, df in IDdf.groupby('day'):
        length.append(len(df))
        full.append(len(df[df['docks_available'] == 0]))
        almost_full.append(len(df[df['docks_available'] <= 3]))
        mask = (df['docks_available'] <= 3) | (df['bikes_available'] <= 3)
        normal.append(len(df[~mask]))
        almost_empty.append(len(df[df['bikes_available'] <= 3]))
        empty.append(len(df[df['bikes_available'] == 0]))

    length = np.array(length)
    full = np.array(full)
    almost_full = np.array(almost_full)
    normal = np.array(normal)
    almost_empty = np.array(almost_empty)
    empty = np.array(empty)

    plt.figure(figsize=(10, 6))

    bars = [np.mean(full/length*100), np.mean(almost_full/length*100), np.mean(normal/length*100), np.mean(almost_empty/length*100), np.mean(empty/length*100)]
    names = ['full', 'almost_full', 'normal', 'almost_empty', 'empty']

    plt.bar(names, bars)
    plt.ylabel('Percentage of time')

    if not os.path.exists(f"../../Results/Occupation/"):
        os.makedirs(f"../../Results/Occupation/")
    plt.savefig(f"../../Results/Occupation/Station_{ID}.jpg", bbox_inches='tight')
    plt.close()

# Count sequences [full, increase] -> [AlmostFull]

In [3]:
interval = 15

status_df["window"] = np.array(status_df['time'].values, dtype=f'datetime64[{interval}m]')

In [4]:
interest_df = status_df[status_df.docks_available < 7]

In [8]:
interest_df

,station_id,bikes_available,docks_available,time,window
10816,2,21,6,2013-09-06 21:16:01,1972-11-29 15:49:00
10817,2,21,6,2013-09-06 21:17:01,1972-11-29 15:49:00
10818,2,21,6,2013-09-06 21:18:01,1972-11-29 15:49:00
10819,2,21,6,2013-09-06 21:19:02,1972-11-29 15:49:00
10820,2,21,6,2013-09-06 21:20:02,1972-11-29 15:49:00
...,...,...,...,...,...
71976999,84,9,6,2015-08-31 08:49:02,1973-01-16 21:23:00
71977000,84,9,6,2015-08-31 08:50:02,1973-01-16 21:23:00
71977001,84,9,6,2015-08-31 08:51:02,1973-01-16 21:23:00
71977002,84,9,6,2015-08-31 08:52:02,1973-01-16 21:23:00


In [23]:
(interest_df['window'].iloc[15] - interest_df['window'].iloc[1]).seconds

60

In [29]:
incr_decr_th = 0
patterns = []


for i, df in interest_df.groupby(['station_id', 'window']):
    hasFull = False
    hasAlmostFull = False
    hasIncrease = False

    if len(df[df.docks_available==0]) > 0:
        hasFull=True
    if len(df[(df.docks_available < 3) & (df.docks_available>0)]) > 0:
        hasAlmostFull=True

    min_val = df['bikes_available'].values[0]
    for el in df['bikes_available'][1:].values:
        if el < min_val:
            min_val = el
        if (el - min_val) > incr_decr_th:
            hasIncrease = True
    patterns.append((i[0], i[1], hasIncrease, hasAlmostFull, hasFull))

KeyboardInterrupt: 

In [30]:
len(patterns)

1176992

In [31]:
df = pd.DataFrame(patterns)

In [34]:
min(df[1])

Timestamp('1972-11-29 02:24:00')

In [36]:
occurrencies = []
past = False

for i, d in df.groupby(0):
    old_window = datetime(1972, 11, 28, 0, 0)
    for _, el in d.iterrows():
        if(el[1] - old_window).seconds == 60:
            if past and el[3]:
                occurrencies.append(1)
            if past:
                occurrencies.append(0)
        past = el[2] & el[4]
        old_window = el[1]

In [37]:
print(np.sum(occurrencies)/len(occurrencies))
print(len(occurrencies))

0.39794721407624634
17050
